In [4]:
from PIL import Image
import pytesseract, re
import cv2
import numpy as np
from fuzzywuzzy import fuzz


def find_name_dob(text):
    dob_idx=-1
    father_name_idx=-1
    name_idx=-1
    account_idx=-1
    
    flag=0
    for i in range(len(text)):
        word_list=text[i].split(' ')
        for s in range(len(word_list)):
            try:
                word_list[s]=word_list[s].split('/')[1]
            except Exception:
                continue
                
        for s in range(len(word_list)):
            try:
                word_list[s]=word_list[s].split('(')[1]
            except Exception:
                continue
        
        for j in range(len(word_list)):
            
            #finding name keyword
            if fuzz.ratio(word_list[j].lower(),'name') >=65:
                if j-1>=0:
                    if fuzz.ratio(word_list[j-1].lower(),'father')<20 or fuzz.ratio(word_list[j-1].lower(),'mother')<20:
                        name_idx=i
                else:
                    name_idx=i
                        
            #finding father name keyword
            if fuzz.ratio(word_list[j].lower(),'name') >=65:
                if j-1>=0:
                    if fuzz.ratio(word_list[j-1].lower(),'father')>60 :
                        father_name_idx=i
                        
            #finding Date_of_birth keyword
            if fuzz.ratio(word_list[j].lower(),'date') >=65:
                if(j+2<len(word_list)):
                    if fuzz.ratio(word_list[j+1].lower(),'of') >=45:
                        if fuzz.ratio(word_list[j+2].lower(),'birth') >=55:
                            dob_idx=i
    

    if name_idx==-1 and dob_idx==-1 and name_idx==-1:
        
        for i in range(len(text)):
            word_list=text[i].split(' ')
            for s in range(len(word_list)):
                try:
                    word_list[s]=word_list[s].split('/')[1]
                except Exception:
                    continue
            for s in range(len(word_list)):
                try:
                    word_list[s]=word_list[s].split('(')[1]
                except Exception:
                    continue
            for j in range(len(word_list)):
                if fuzz.ratio(word_list[j].lower(),'number') >=65:
                    if j-2>=0:
                        if fuzz.ratio(word_list[j-1].lower(),'account') >=45:
                            if fuzz.ratio(word_list[j-2].lower(),'permanent') >=55:
                                account_idx=i
                                
        for a in range(account_idx-1,0,-1):
            
            if text[a]!='' or text[a]!=' ' or text[a]!='\n':
                dob_idx=a-1
                break
                
                
        #dob_idx=account_idx-1
        father_name_idx=dob_idx-1
        for a in range(father_name_idx-1,0,-1):
            
            if text[a]!='' or text[a]!=' ' or text[a]!='\n':
                name_idx=a-1
                break
        
        
        #name_idx=father_name_idx-1
        if father_name_idx!=-1 and name_idx!=-1 and dob_idx!=-1:
            print('Name: ',text[name_idx])
            print('Father Name: ',text[father_name_idx])
            print('DOB: ',text[dob_idx])
    else:
        print(text[name_idx]+' : '+text[name_idx+1])
        print(text[father_name_idx]+' : '+text[father_name_idx+1])
        print(text[dob_idx]+' : '+text[dob_idx+1])















#noise removal
#--------------------------
# Read image with opencv
img = cv2.imread("./data/pan card/pan5.jpeg")

# Convert to gray
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply dilation and erosion to remove some noise
kernel = np.ones((1, 1), np.uint8)
img = cv2.dilate(img, kernel, iterations=1)
cv2.imwrite("./dilate/dilate.jpg", img)

img = cv2.erode(img, kernel, iterations=1)
cv2.imwrite("./erode/erode.jpg", img)
# Write image after removed noise
cv2.imwrite("removed_noise.jpg", img)

#  Apply threshold to get image with only black and white
img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 10)


# Write the image after apply opencv to do some ...
cv2.imwrite("thres.jpg", img)


f = "thres.jpg"

config="--psm 01"
text = pytesseract.image_to_string(Image.open(f),lang='eng',config=config)
text=text.split('\n')
#print('t= ',text)
find_name_dob(text)



# for i in range(len(text)):
#     print(i,text[i])

Name:  ) SUTAPA MONDAL
Father Name:  | ‘BECHARAM MONDAL
DOB:  1511171988
